In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
'''
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
'''
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
'''
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
'''
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
print(os.listdir("/kaggle/input/notebookf30568c6d4"))

In [ ]:
import tensorflow as tf
from tensorflow import keras
print(tf.__version__)
print(keras.__version__)

In [ ]:
import pandas as pd
import numpy as np
import itertools
import os
import sys
from prettytable import PrettyTable
import pickle
from random import shuffle
from sklearn.model_selection import train_test_split
import multiprocessing
from multiprocessing.pool import ThreadPool
from tqdm import tqdm_notebook
print(multiprocessing.cpu_count()," CPU cores")

import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams["axes.grid"] = False

from sklearn.metrics import confusion_matrix, cohen_kappa_score,accuracy_score
from sklearn.preprocessing import MultiLabelBinarizer

from PIL import Image
import cv2
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import applications
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers,Model,Sequential
from tensorflow.keras.layers import Input,GlobalAveragePooling2D,Dropout,Dense,Activation,BatchNormalization,GlobalMaxPooling2D,concatenate,Flatten
from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau,Callback
from tensorflow.keras.initializers import random_normal
from tensorflow.keras.models import load_model
from tensorflow.keras.losses import binary_crossentropy,categorical_crossentropy,mean_squared_error
from tensorflow.keras import backend as K


In [ ]:
file = open('/kaggle/input/notebookf30568c6d4/df_train_train', 'rb')
df_train_train = pickle.load(file)
file.close()

file = open('/kaggle/input/notebookf30568c6d4/df_train_test', 'rb')
df_train_test = pickle.load(file)
file.close()

In [ ]:
FACTOR = 4
BATCH_SIZE = 8 * FACTOR
IMG_SIZE = 512
EPOCHS = 20
WARMUP_EPOCHS = 5
LEARNING_RATE = 1e-4 * FACTOR
WARMUP_LEARNING_RATE = 1e-3 * FACTOR
HEIGHT = 320
WIDTH = 320
CANAL = 3
N_CLASSES = df_train_train['diagnosis'].nunique()
ES_PATIENCE = 5
RLROP_PATIENCE = 3
DECAY_DROP = 0.5

LR_WARMUP_EPOCHS_1st = 2
LR_WARMUP_EPOCHS_2nd = 5
STEP_SIZE = len(df_train_train) // BATCH_SIZE
TOTAL_STEPS_1st = WARMUP_EPOCHS * STEP_SIZE
TOTAL_STEPS_2nd = EPOCHS * STEP_SIZE
WARMUP_STEPS_1st = LR_WARMUP_EPOCHS_1st * STEP_SIZE
WARMUP_STEPS_2nd = LR_WARMUP_EPOCHS_2nd * STEP_SIZE

In [ ]:
def multiple_outputs(generator,dataframe, image_dir, batch_size, height,width, subset):
    gen = generator.flow_from_dataframe(
        dataframe = dataframe,
        x_col = "file_name",
        y_col = "diagnosis",
        directory = image_dir,
        target_size=(height, width),
        batch_size=batch_size,
        class_mode='categorical',
        subset=subset)
    
    mlb = MultiLabelBinarizer(classes = range(N_CLASSES))
    
    while True:
        gnext = gen.next()
        yield gnext[0], [np.argmax(gnext[1],axis = -1),gnext[1],mlb.fit_transform([list(range(x+1)) for x in np.argmax(gnext[1],axis = -1)])]

In [ ]:
train_datagen=ImageDataGenerator(rescale=1./255, rotation_range=360,brightness_range=[0.5, 1.5],
                                     zoom_range=[1, 1.2],zca_whitening=True,horizontal_flip=True,
                                     vertical_flip=True,fill_mode='constant',cval=0.,validation_split = 0.2)

train_generator = multiple_outputs(generator = train_datagen,dataframe = df_train_train,
                                   image_dir="/kaggle/input/notebookf30568c6d4/train_images_resized_preprocessed/",
                                   batch_size=BATCH_SIZE,height = HEIGHT,width = WIDTH,
                                   subset='training')
valid_generator=multiple_outputs(generator = train_datagen,dataframe = df_train_train,
                                   image_dir="/kaggle/input/notebookf30568c6d4/train_images_resized_preprocessed/",
                                   batch_size=BATCH_SIZE,height = HEIGHT,width = WIDTH,
                                   subset='validation')
     
# valid_generator = multiple_outputs(generator = train_datagen,dataframe = df_train_test,
#                                    image_dir="/kaggle/input/notebookf30568c6d4/test_images_resized_preprocessed/",
#                                    batch_size=BATCH_SIZE,height = HEIGHT,width = WIDTH,
#                                    subset='validation')

In [ ]:
input_tensor = Input(shape=(HEIGHT, WIDTH, CANAL))
base_model = applications.ResNet50(weights=None, include_top=False,input_tensor=input_tensor)
base_model.load_weights('/kaggle/input/resnet50/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5')


x1 = GlobalAveragePooling2D()(base_model.output)
x1 = BatchNormalization()(x1)

x2 = GlobalMaxPooling2D()(base_model.output)
x2 = BatchNormalization()(x2)

x = concatenate([x1,x2])

# Regression Head
xr = Dense(2048, activation='relu')(x)
xr = Dropout(0.5)(xr)
xr = Dense(1,activation = 'linear',name = 'regression_output')(xr)

# Classification Head
xc = Dense(2048, activation='relu')(x)
xc = Dropout(0.5)(xc)
xc = Dense(N_CLASSES,activation = 'softmax',name = 'classification_output')(xc)

# Ordinal Regression Head
xo = Dense(2048, activation='relu')(x)
xo = Dropout(0.5)(xo)
xo = Dense(N_CLASSES,activation = 'softmax',name = 'ordinal_regression_output')(xo)

model = Model(inputs = [input_tensor], outputs = [xr,xc,xo])

In [ ]:
for layer in model.layers:
    layer.trainable = True
# model.summary()

In [ ]:
STEP_SIZE_TRAIN = len(df_train_train)//BATCH_SIZE
STEP_SIZE_VALID = len(df_train_test)//BATCH_SIZE
print(STEP_SIZE_TRAIN,STEP_SIZE_VALID)

In [ ]:
def cosine_decay_with_warmup(global_step,
                             learning_rate_base,
                             total_steps,
                             warmup_learning_rate=0.0,
                             warmup_steps=0,
                             hold_base_rate_steps=0):
    """
    Cosine decay schedule with warm up period.
    In this schedule, the learning rate grows linearly from warmup_learning_rate
    to learning_rate_base for warmup_steps, then transitions to a cosine decay
    schedule.
    :param global_step {int}: global step.
    :param learning_rate_base {float}: base learning rate.
    :param total_steps {int}: total number of training steps.
    :param warmup_learning_rate {float}: initial learning rate for warm up. (default: {0.0}).
    :param warmup_steps {int}: number of warmup steps. (default: {0}).
    :param hold_base_rate_steps {int}: Optional number of steps to hold base learning rate before decaying. (default: {0}).
    :param global_step {int}: global step.
    :Returns : a float representing learning rate.
    :Raises ValueError: if warmup_learning_rate is larger than learning_rate_base, or if warmup_steps is larger than total_steps.
    """

    if total_steps < warmup_steps:
        raise ValueError('total_steps must be larger or equal to warmup_steps.')
    learning_rate = 0.5 * learning_rate_base * (1 + np.cos(
        np.pi *
        (global_step - warmup_steps - hold_base_rate_steps
         ) / float(total_steps - warmup_steps - hold_base_rate_steps)))
    if hold_base_rate_steps > 0:
        learning_rate = np.where(global_step > warmup_steps + hold_base_rate_steps,
                                 learning_rate, learning_rate_base)
    if warmup_steps > 0:
        if learning_rate_base < warmup_learning_rate:
            raise ValueError('learning_rate_base must be larger or equal to warmup_learning_rate.')
        slope = (learning_rate_base - warmup_learning_rate) / warmup_steps
        warmup_rate = slope * global_step + warmup_learning_rate
        learning_rate = np.where(global_step < warmup_steps, warmup_rate,
                                 learning_rate)
    return np.where(global_step > total_steps, 0.0, learning_rate)


class WarmUpCosineDecayScheduler(Callback):
    """Cosine decay with warmup learning rate scheduler"""

    def __init__(self,
                 learning_rate_base,
                 total_steps,
                 global_step_init=0,
                 warmup_learning_rate=0.0,
                 warmup_steps=0,
                 hold_base_rate_steps=0,
                 verbose=0):
        """
        Constructor for cosine decay with warmup learning rate scheduler.
        :param learning_rate_base {float}: base learning rate.
        :param total_steps {int}: total number of training steps.
        :param global_step_init {int}: initial global step, e.g. from previous checkpoint.
        :param warmup_learning_rate {float}: initial learning rate for warm up. (default: {0.0}).
        :param warmup_steps {int}: number of warmup steps. (default: {0}).
        :param hold_base_rate_steps {int}: Optional number of steps to hold base learning rate before decaying. (default: {0}).
        :param verbose {int}: quiet, 1: update messages. (default: {0}).
        """

        super(WarmUpCosineDecayScheduler, self).__init__()
        self.learning_rate_base = learning_rate_base
        self.total_steps = total_steps
        self.global_step = global_step_init
        self.warmup_learning_rate = warmup_learning_rate
        self.warmup_steps = warmup_steps
        self.hold_base_rate_steps = hold_base_rate_steps
        self.verbose = verbose
        self.learning_rates = []

    def on_batch_end(self, batch, logs=None):
        self.global_step = self.global_step + 1
        lr = K.get_value(self.model.optimizer.lr)
        self.learning_rates.append(lr)

    def on_batch_begin(self, batch, logs=None):
        lr = cosine_decay_with_warmup(global_step=self.global_step,
                                      learning_rate_base=self.learning_rate_base,
                                      total_steps=self.total_steps,
                                      warmup_learning_rate=self.warmup_learning_rate,
                                      warmup_steps=self.warmup_steps,
                                      hold_base_rate_steps=self.hold_base_rate_steps)
        K.set_value(self.model.optimizer.lr, lr)
        if self.verbose > 0:
            print('\nBatch %02d: setting learning rate to %s.' % (self.global_step + 1, lr))

In [ ]:
cosine_lr = WarmUpCosineDecayScheduler(learning_rate_base = LEARNING_RATE,
                                       total_steps=TOTAL_STEPS_1st,
                                       warmup_learning_rate=0.0,
                                       warmup_steps=TOTAL_STEPS_1st,
                                       hold_base_rate_steps=(2 * STEP_SIZE))

callback_list = [cosine_lr]

In [ ]:
model.compile(optimizer = optimizers.SGD(lr=LEARNING_RATE),
              loss={'regression_output': 'mean_absolute_error', 
                    'classification_output': 'categorical_crossentropy',
                    'ordinal_regression_output' : 'binary_crossentropy'
                    },
              metrics = ['accuracy'])

In [ ]:
history = model.fit_generator(generator=train_generator,
                              steps_per_epoch=STEP_SIZE_TRAIN,
                              validation_data=valid_generator,
                              validation_steps=STEP_SIZE_VALID,
                              epochs=20,
                              callbacks = callback_list,
                              verbose=1)

model.save("/kaggle/working/model_pre_training.h5")
# f = open("/kaggle/working/history_pre_training","wb")
# pickle.dump(history,f)
# f.close()

In [ ]:
f = open("/kaggle/working/history_pre_training","wb")
pickle.dump(history.history,f)
f.close()

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(history.history['regression_output_loss'])
plt.plot(history.history['val_regression_output_loss'])
plt.title('Regression Model Loss - Pre Training')
plt.ylabel('Loss (MAE)')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='best')
plt.xticks(range(1,21))
plt.gca().ticklabel_format(axis='both', style='plain', useOffset=False)
plt.show()

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(history.history['classification_output_loss'])
plt.plot(history.history['val_classification_output_loss'])
plt.title('Classification Model Loss - Pre Training')
plt.ylabel('Loss (Categorical Cross Entropy)')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='best')
plt.xticks(range(1,21))
plt.gca().ticklabel_format(axis='both', style='plain', useOffset=False)
plt.show()

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(history.history['ordinal_regression_output_loss'])
plt.plot(history.history['val_ordinal_regression_output_loss'])
plt.title('Ordinal Regression Model Loss - Pre Training')
plt.ylabel('Loss (Binary Cross Entropy)')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='best')
plt.xticks(range(1,21))
plt.gca().ticklabel_format(axis='both', style='plain', useOffset=False)
plt.show()

In [ ]:
for layer in model.layers:
    layer.trainable = False

for i in range(-14,0):
  model.layers[i].trainable = True

# model.summary()

In [ ]:
# ref - https://github.com/umbertogriffo/focal-loss-keras/blob/master/losses.py

'''Below Functions create custom loss functions - Categorical Focal Loss, Binary Focal Loss'''

def binary_focal_loss(gamma=2., alpha=.25):
    """
    Binary form of focal loss.
      FL(p_t) = -alpha * (1 - p_t)**gamma * log(p_t)
      where p = sigmoid(x), p_t = p or 1 - p depending on if the label is 1 or 0, respectively.
    References:
        https://arxiv.org/pdf/1708.02002.pdf
    Usage:
     model.compile(loss=[binary_focal_loss(alpha=.25, gamma=2)], metrics=["accuracy"], optimizer=adam)
    """
    def binary_focal_loss_fixed(y_true, y_pred):
        """
        :param y_true: A tensor of the same shape as `y_pred`
        :param y_pred:  A tensor resulting from a sigmoid
        :return: Output tensor.
        """
        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))

        epsilon = K.epsilon()
        # clip to prevent NaN's and Inf's
        pt_1 = K.clip(pt_1, epsilon, 1. - epsilon)
        pt_0 = K.clip(pt_0, epsilon, 1. - epsilon)

        return -K.mean(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1)) \
               -K.mean((1 - alpha) * K.pow(pt_0, gamma) * K.log(1. - pt_0))

    return binary_focal_loss_fixed

In [ ]:
def categorical_focal_loss(gamma=2., alpha=.25):
    """
    Softmax version of focal loss.
           m
      FL = ∑  -alpha * (1 - p_o,c)^gamma * y_o,c * log(p_o,c)
          c=1
      where m = number of classes, c = class and o = observation
    Parameters:
      alpha -- the same as weighing factor in balanced cross entropy
      gamma -- focusing parameter for modulating factor (1-p)
    Default value:
      gamma -- 2.0 as mentioned in the paper
      alpha -- 0.25 as mentioned in the paper
    References:
        Official paper: https://arxiv.org/pdf/1708.02002.pdf
        https://www.tensorflow.org/api_docs/python/tf/keras/backend/categorical_crossentropy
    Usage:
     model.compile(loss=[categorical_focal_loss(alpha=.25, gamma=2)], metrics=["accuracy"], optimizer=adam)
    """
    def categorical_focal_loss_fixed(y_true, y_pred):
        """
        :param y_true: A tensor of the same shape as `y_pred`
        :param y_pred: A tensor resulting from a softmax
        :return: Output tensor.
        """

        # Scale predictions so that the class probas of each sample sum to 1
        y_pred /= K.sum(y_pred, axis=-1, keepdims=True)

        # Clip the prediction value to prevent NaN's and Inf's
        epsilon = K.epsilon()
        y_pred = K.clip(y_pred, epsilon, 1. - epsilon)

        # Calculate Cross Entropy
        cross_entropy = -y_true * K.log(y_pred)

        # Calculate Focal Loss
        loss = alpha * K.pow(1 - y_pred, gamma) * cross_entropy

        # Compute mean loss in mini_batch
        return K.mean(loss, axis=1)

    return categorical_focal_loss_fixed

In [ ]:
model.compile(optimizer = optimizers.Adam(lr=WARMUP_LEARNING_RATE),
              loss={'regression_output': mean_squared_error, 
                    'classification_output': categorical_focal_loss(alpha=.25, gamma=2) ,
                    'ordinal_regression_output' : binary_focal_loss(alpha=.25, gamma=2)
                    },
              metrics = ['accuracy'])

history = model.fit_generator(generator=train_generator,
                              steps_per_epoch=STEP_SIZE_TRAIN,
                              validation_data=valid_generator,
                              validation_steps=STEP_SIZE_VALID,
                              epochs=5,
                              callbacks = callback_list,
                              verbose=1).history

model.save("/kaggle/working/model_main_training.h5")
# f = open("/kaggle/working/history_main_training","wb")
# pickle.dump(history.history,f)
# f.close()

In [ ]:
for layer in model.layers:
    layer.trainable = True
# model.summary()

In [ ]:
model.compile(optimizer = optimizers.Adam(lr=LEARNING_RATE),
              loss={'regression_output': mean_squared_error, 
                    'classification_output': categorical_focal_loss(alpha=.25, gamma=2) ,
                    'ordinal_regression_output' : binary_focal_loss(alpha=.25, gamma=2)
                    },
              metrics = ['accuracy'])

history = model.fit_generator(generator=train_generator,
                              steps_per_epoch=STEP_SIZE_TRAIN,
                              validation_data=valid_generator,
                              validation_steps=STEP_SIZE_VALID,
                              epochs=45,
                              callbacks = callback_list,
                              verbose=1).history

model.save("/kaggle/working/model_main_training.h5")
# f = open("/kaggle/working/history_main_training","wb")
# pickle.dump(history.history,f)
# f.close()

In [ ]:
f = open("/kaggle/working/history_main_training","wb")
pickle.dump(history,f)
f.close()

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(history['regression_output_loss'])
plt.plot(history['val_regression_output_loss'])
plt.title('Regression Model Loss - Main Training')
plt.ylabel('Loss (MSE)')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='best')
plt.xticks(range(1,46))
plt.gca().ticklabel_format(axis='both', style='plain', useOffset=False)
plt.show()

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(history['classification_output_loss'])
plt.plot(history['val_classification_output_loss'])
plt.title('Classification Model Loss - Main Training')
plt.ylabel('Loss (Categorical Focal Loss)')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='best')
plt.xticks(range(1,46))
plt.gca().ticklabel_format(axis='both', style='plain', useOffset=False)
plt.show()

In [ ]:

plt.figure(figsize=(15,5))
plt.plot(history['ordinal_regression_output_loss'])
plt.plot(history['val_ordinal_regression_output_loss'])
plt.title('Ordinal Regression Model Loss - Main Training')
plt.ylabel('Loss (Binary Focal Loss)')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='best')
plt.xticks(range(1,46))
plt.gca().ticklabel_format(axis='both', style='plain', useOffset=False)
plt.show()

In [ ]:
print('hello world')